In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(host='localhost',port=9200)

Elasticsearch doc:https://www.elastic.co/guide/en/elasticsearch/reference/current/analyzer.html

Analyzers can be specified per-query, per-field or per-index.
<br> 
<b> Types of Analysers: </b>
* Standard Analyzer
* Simple Analyzer
* Whitespace Analyzer
* Stop Analyzer
* Keyword Analyzer
* Pattern Analyzer
* Language Analyzers
* Fingerprint Analyzer

#### NOTE ON FINGERPRINT ALGORITHM:
It implements fingerprint algorithm which 
* remove leading and trailing whitespace
* change all characters to their lowercase representation
* remove all punctuation and control characters
* normalize extended western characters to their ASCII representation (for example "gödel" → "godel")
* split the string into whitespace-separated tokens
* sort the tokens and remove duplicates and join the tokens back together <br>
Hence, `Mishra Amogh` will be represented as `amogh mishra`

In [ ]:
es.indices.analyze(body={
  "analyzer" : "standard",
  "text" : ["HELLO today is A GREAT DAY"]
})

In [24]:
analyzer = ['standard','simple','whitespace','stop','keyword','pattern','fingerprint']

for analyze in analyzer:
    res = es.indices.analyze(body={
      "analyzer" : analyze,
      "text" : ["HELLO WORLD. Today is the 2nd day of the week!!!!     it is Monday."]
    })
    print("======",analyze,"========")
    for i in res['tokens']:
        print(i['token'])
    print("\n")

('======', 'standard', '========')
hello
world
today
is
the
2nd
day
of
the
week
it
is
monday


('======', 'simple', '========')
hello
world
today
is
the
nd
day
of
the
week
it
is
monday


('======', 'whitespace', '========')
HELLO
WORLD.
Today
is
the
2nd
day
of
the
week!!!!
it
is
Monday.


('======', 'stop', '========')
hello
world
today
nd
day
week
monday


('======', 'keyword', '========')
HELLO WORLD. Today is the 2nd day of the week!!!!     it is Monday.


('======', 'pattern', '========')
hello
world
today
is
the
2nd
day
of
the
week
it
is
monday


('======', 'fingerprint', '========')
2nd day hello is it monday of the today week world




In [32]:
body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "urls": {
            "properties": {
                "text": {
                    "type": "text",
                    "fields": {
                      "english": { 
                        "type":     "text",
                        "analyzer": "whitespace"
                      }
                    }
                }
            }
        }
     }
}
# create index
es.indices.create(index="chapter9-analyzer", ignore=400, body=body)


{u'acknowledged': True,
 u'index': u'chapter9-analyzer',
 u'shards_acknowledged': True}

In [33]:
res = es.indices.analyze(index="chapter9-analyzer", body = {
  "field": "text.whitespace",
  "text": "The quick Brown Foxes   ."
})
for i in res['tokens']:
    print(i['token'])

the
quick
brown
foxes


In [31]:
es.indices.delete(index="chapter9-analyzer")

{u'acknowledged': True}